In [2]:
import torch
print("CUDA available:", torch.cuda.is_available())

CUDA available: True


In [3]:
import numpy as np
import torch
from torch import nn
from torch.utils.data import Dataset, DataLoader, random_split
from PIL import Image
from torch.nn import functional as F
import torchvision
from torchvision import datasets, transforms
import pandas as pd
from torch.optim.lr_scheduler import StepLR
import matplotlib.pyplot as plt
from tqdm import tqdm  # Import tqdm for progress bar
import py7zr
from io import BytesIO ## this too
import os

In [4]:
# Define device (GPU/CPU)
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [5]:
model = nn.Sequential(
    # Conv2d( in_channels, out_channels, kernel_size) B,3,32,32
    nn.Conv2d(3, 64, 3, stride= 1, padding=1),  nn.BatchNorm2d(64), nn.LeakyReLU(0.1) , nn.MaxPool2d(2, 2), # B,32,16,16
    nn.Conv2d(64, 128, 3, stride= 1, padding=1), nn.BatchNorm2d(128), nn.LeakyReLU(0.1), nn.MaxPool2d(2, 2), # B,32,8,8
    nn.Conv2d(128, 256, 3, stride= 1, padding=1), nn.BatchNorm2d(256), nn.LeakyReLU(0.1), nn.MaxPool2d(2, 2), # B,32,4,4
    nn.Conv2d(256, 512, 3, stride= 1, padding=1 ), nn.BatchNorm2d(512), nn.LeakyReLU(0.1),
    nn.AdaptiveAvgPool2d(1), ##like in ResNET
    nn.Flatten(),
    nn.Linear(512, 512), nn.BatchNorm1d(512), nn.LeakyReLU(0.1),
    nn.Linear(512, 10)
)

optim = torch.optim.Adam(model.parameters(), lr=0.0003, weight_decay=0.0005)
scheduler = StepLR(optim, step_size=10, gamma=0.5)

In [6]:
# Define transformations
transform_train = transforms.Compose([
    transforms.RandomHorizontalFlip(0.5),
    transforms.RandomRotation(15),
    transforms.ColorJitter(0.1, 0.1, 0.1),
    transforms.RandomCrop(32, padding=4),
    transforms.ToTensor(),
    transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2470, 0.2435, 0.2616)),
    transforms.RandomErasing(p=0.5,scale=(0.02, 0.1),value=1.0, inplace=False),
])

transform_test = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2470, 0.2435, 0.2616))
])

batch_size=64

In [7]:
## Train with the good old torchvision set

cifar10 = torchvision.datasets.CIFAR10(root='./data', download=True, train=True, transform=transform_train)
cifar10_test = torchvision.datasets.CIFAR10(root='./data', download=True, train=False, transform=transform_test)

batch_size = 64 ## increase
n = batch_size
dataloader = DataLoader(cifar10, batch_size=batch_size, shuffle=True, num_workers=2)
dataloader_test = DataLoader(cifar10_test, batch_size=batch_size, shuffle=False, num_workers=2) 

Files already downloaded and verified
Files already downloaded and verified


In [8]:
## xavier init
def init_weights_xavier(m):
    if isinstance(m, nn.Conv2d) or isinstance(m, nn.Linear):
        nn.init.xavier_normal_(m.weight)  # Apply Xavier normal initialization
        if m.bias is not None:
            nn.init.constant_(m.bias, 0)
    elif isinstance(m, nn.BatchNorm2d) or isinstance(m, nn.BatchNorm1d):
        nn.init.constant_(m.weight, 1)
        nn.init.constant_(m.bias, 0)

model.apply(init_weights_xavier)

Sequential(
  (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (2): LeakyReLU(negative_slope=0.1)
  (3): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (4): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (5): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (6): LeakyReLU(negative_slope=0.1)
  (7): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (8): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (9): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (10): LeakyReLU(negative_slope=0.1)
  (11): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (12): Conv2d(256, 512, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (13): BatchNorm2d(512, eps=1e-05, momentum=0.1, affine=True, track_

In [9]:
i = 0
losses = []
steps = []
losses_t = []

In [10]:
for epoch in range(32):
    for batch in dataloader:
        i +=1
        x = batch[0].to('cuda')
        y = batch[1].to('cuda')
        model = model.to('cuda')

        logits = model(x)
        loss = F.cross_entropy(logits,y)
        optim.zero_grad()

        loss.backward()
        optim.step()

        losses.append(loss.item())
        losses_t.append(loss.detach())
        steps.append(i)

        if i % 1000 == 0:
            print(loss)
    scheduler.step()

tensor(1.2896, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.9978, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.8442, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.7868, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.8090, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.8523, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.7030, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.6675, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.5619, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.5129, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.6790, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.6121, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.6055, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.4129, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.4569, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.3794, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.4395, device='cuda:0', grad_fn=

In [28]:
# plt.plot(steps, losses)

In [11]:
total_correct = 0
total_predictions = 0

# Loop over training dataset
for x_batch, y_batch in dataloader:
    logits = model(x_batch.to("cuda")) # Forward pass on the mini-batch
    loss = F.cross_entropy(logits.cpu(), y_batch) # Compute loss

    # Calculate predictions for the batch
    pred_labels = torch.max(logits, dim=1).indices

    # Update total correct predictions and total predictions
    total_correct += (y_batch == pred_labels.cpu()).sum().item()
    total_predictions += y_batch.size(0)

# Calculate overall accuracy
overall_accuracy = total_correct / total_predictions
print(f"Training Accuracy: {overall_accuracy}")

Training Accuracy: 0.87712


In [12]:
total_correct = 0
total_predictions = 0

# Loop over testing dataset
for x_batch, y_batch in dataloader_test:
    logits = model(x_batch.to("cuda")) # Forward pass on the mini-batch
    loss = F.cross_entropy(logits.cpu(), y_batch) # Compute loss

    # Calculate predictions for the batch
    pred_labels = torch.max(logits, dim=1).indices

    # Update total correct predictions and total predictions
    total_correct += (y_batch == pred_labels.cpu()).sum().item()
    total_predictions += y_batch.size(0)

# Calculate overall accuracy and print it out.
overall_accuracy = total_correct / total_predictions
#DON'T FORGET TO PRINT OUT YOUR TESTING ACCURACY
print(f"Testing Accuracy: {overall_accuracy}")

Testing Accuracy: 0.8536


In [24]:
import os

# 获取当前 notebook 文件所在的路径
current_dir = os.path.dirname(os.path.abspath("__file__"))

# 设置工作目录为当前文件夹
os.chdir(current_dir)
print("工作目录已设置为：", current_dir)


工作目录已设置为： C:\Users\Frank\Desktop\Deep learning fundational\Assignment 2


In [27]:
# Create submission file with predictions for test images from .7z archive
test_filenames = []
test_images = []

with py7zr.SevenZipFile('cifar-10/test.7z', mode='r') as z:
    for name, file in z.readall().items():
        if name.endswith('.png'):
            img = Image.open(BytesIO(file.read()))
            test_images.append(transform_test(img))
            test_filenames.append(name)

test_images = torch.stack(test_images)
test_loader = DataLoader(test_images, batch_size=batch_size, shuffle=False)

# Prediction and CSV creation
classes = ('airplane', 'automobile', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck')

result = []
with torch.no_grad():
    model.eval()
    for inputs in tqdm(test_loader):
        inputs = inputs.to(device)
        outputs = model(inputs)
        _, predicted = outputs.max(1)
        result.extend(predicted.cpu().numpy())

# Create submission DataFrame
submission_df = pd.DataFrame({
    'id': [os.path.basename(f).replace('.png', '') for f in test_filenames],  # Remove .png from filenames
    'label': [classes[label] for label in result]
})

# Save submission file
submission_df.to_csv('submission.csv', index=False)

print("Submission file created successfully!")

100%|██████████| 4688/4688 [00:08<00:00, 537.04it/s]


Submission file created successfully!
